# 生成器 generator
一种特殊的迭代器 iterator，在需要时动态生成值，实现“惰性计算”

## 列表推导式与生成器表达式
- 大型数据集或流数据时，生成器表达式更优，节省内存
  - 如处理文件字节长度 sum(x for x in (len(line) for line in open(filename)))
- 小型数据集，列表推导式更快，可一次性生成所有值

In [ ]:
## 1. 列表推导式 <class 'list'>
lst_generator = [0 for _ in range(10)] # 10 个元素为0 的列表
print(f'lst_generator: {lst_generator}, 其类型为{type(lst_generator)}')

## 2. 生成器表达式 <class 'generator'>，与列表推导式类似，但返回的是生成器对象而不是列表
generator_expression = (0 for _ in range(10))
print(f'generator_expression: {generator_expression}, 其类型为{type(generator_expression)}')

print(next(generator_expression))
print(next(generator_expression))
# 注意 next 次数不可超过 generator_expression 的长度，否则抛错 StopIteration

## 3. 生成器函数的使用
# 使用 yield 返回值，不是 return
# 使用生成器函数会返回一个迭代器，可以使用 next() 函数获取值

def my_generator():
    for i in range(10):
        yield i

gen = my_generator()
print(next(gen)) # 0
print(next(gen)) # 1

# 生成器双向通信：不仅可以向调用者返回值，还可以从调用者接收值（PS：进阶：还支持 throw 和 close 方法）
def send_generator_for_plus():
    x = yield
    while True:
        x = yield x + x

sg = send_generator_for_plus()
next(sg)
print(sg.send(10))
print(sg.send(3))





lst_generator: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 其类型为<class 'list'>
generator_expression: <generator object <genexpr> at 0x10f463510>, 其类型为<class 'generator'>
0
0
0
1
20
6


## \_\_slots\_\_ 限制对象属性
自定义对象的初始化，重载 \_\_new\_\_ 而不是单纯的 \_\_init\_\_，实现预分配内存空间
- 注意，对继承的子类无效。子类中定义 slots 后，子类的实例允许的属性范围则是子类+父类的 slots。

In [13]:
import sys

class People:
    def __init__(self, name):
        self.name = name

class Human:
    __slots__ = ["name"]

    def __init__(self, name):
        self.name = name

p = People('mj')
h = Human('hello world mj')

p_size = sys.getsizeof(p)
h_size = sys.getsizeof(h)

print(f'p size: {p_size}, h size: {h_size}, cut down {round((p_size - h_size)/p_size, 1) * 100}% memory')


p size: 56, h size: 40, cut down 30.0% memory


# Python 管理内存
https://docs.python.org/zh-tw/3.11/library/tracemalloc.html
- objgraph
- memory_profiler
- tracemalloc

## tracemalloc
tracemalloc 模块是一个用于对 python 已申请的内存快进行 debug 的工具。它能以供一下信息：
- 回溯对象分配内存的位置
- 按文件、行统计 python 的内存块分配情况：内存块总大小、数量以及块平均大小
- 对比两个内存快照的差异，以便排查内存泄露

### 显示前 10 项

In [1]:
import tracemalloc

tracemalloc.start()
a = [i for i in range(10000)]
print('hello world')
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')

print('[Top 10 malloc]')
for stat in top_stats[:10]:
    print(stat)

hello world
[Top 10 malloc]
/var/folders/wy/sv7xp55526xfh1ky9r2fcyy40000gn/T/ipykernel_28679/1615604996.py:4: size=388 KiB, count=9744, average=41 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipykernel/iostream.py:684: size=1122 B, count=7, average=160 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/traitlets/traitlets.py:652: size=768 B, count=1, average=768 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/codeop.py:118: size=669 B, count=8, average=84 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3517: size=304 B, count=1, average=304 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py:313: size=176 B, count=2, average=88 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipykernel/iostream.py:587: size=152 B, count=1, average=152 B
/Library/Frameworks/Python.framework

### 计算差异
获取两个快照并显示差异

In [2]:
import tracemalloc

tracemalloc.start()
a = [i for i in range(5)]
print('hello world')
snapshot1 = tracemalloc.take_snapshot()
b = (i for i in range(2**32))
snapshot2 = tracemalloc.take_snapshot()

top_stats = snapshot2.compare_to(snapshot1, 'lineno')

print('[Top 10 malloc]')
for stat in top_stats[:10]:
    print(stat)

hello world
[Top 10 malloc]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/codeop.py:118: size=1651 B (+430 B), count=22 (+5), average=75 B
/var/folders/wy/sv7xp55526xfh1ky9r2fcyy40000gn/T/ipykernel_28679/796183124.py:7: size=400 B (+400 B), count=3 (+3), average=133 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tracemalloc.py:560: size=632 B (+312 B), count=4 (+2), average=158 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tracemalloc.py:423: size=632 B (+312 B), count=4 (+2), average=158 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/tracemalloc.py:558: size=240 B (+240 B), count=4 (+4), average=60 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py:781: size=288 B (-72 B), count=3 (-1), average=96 B
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/history.py:837: size=0 B (-72 B), count=0 (-1)
/Library/Frameworks/Python

### 获取一个内存块的溯源 code
一段找出程序中最大内存块溯源的代码